In [ ]:
import xpress as xp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Create a problem 

stage_1 = xp.problem(name='F_O stage 1')

In [ ]:
# Import data into panda dataframes

df_rooms = pd.read_excel('Relevant Data Room.xlsx', header=0)
df_courses = pd.read_excel('Course Data.xlsx', header=0)
df_curricula = pd.read_excel('Baseline solution(ACHIEVED DELIVERABLE) and extracted data(ONGOING).xlsx', header=0, sheet_name='Curricula')

df_courses

In [ ]:
# Index sets
courses = list(df_courses['Course'])
lecture_hours = list(df_courses['Lecture - Duration'])
workshop_hours = list(df_courses['Workshop - Duration'])
curricula = list(df_curricula['Code'])
timeslots = [i for i in range(9)]
days = ['M_O', 'T_O', 'W_O', 'TH_O', 'F_O', 'M_E', 'T_E', 'W_E', 'TH_E', 'F_E']
rooms = list(df_rooms['ROOM INDEX'])
room_types = sorted(df_rooms['ROOM LAYOUT'].unique())
class_type = ['L', 'W']
workshop_pattern = [1, 2, 3]

capacities = sorted(np.append(df_rooms['CAP'].unique(), 0))

# Parameters 
#P1: The number of lecture hours for course c 
lecture_hours_dict = dict(zip(courses, lecture_hours))

#P2: The number of workshop hours for course c 
workshop_hours_dict = dict(zip(courses, workshop_hours))


# Create the variables
x = np.array([xp.var( name='x_{0}_{1}_{2}'.format(i, j, k), vartype=xp.binary)
                  for i in courses for j in timeslots for k in class_type], dtype=xp.npvar).reshape(len(courses), len(timeslots), len(class_type))

y = np.array([xp.var( name='x_{0}_{1}_{2}'.format(i, j, k), vartype=xp.binary)
                  for i in courses for j in timeslots for k in capacities], dtype=xp.npvar).reshape(len(courses), len(timeslots), len(capacities))

z = np.array([xp.var( name='z_{0}_{1}'.format(i, j), vartype=xp.binary)
                  for i in courses for j in days], dtype=xp.npvar).reshape(len(courses), len(days))

w = np.array([xp.var( name='w_{0}'.format(i))
                    for i in courses], dtype=xp.npvar)

r = np.array([xp.var( name='r_{0}_{1}'.format(i, j), vartype=xp.binary)
                  for i in curricula for j in timeslots], dtype=xp.npvar).reshape(len(curricula), len(timeslots))

v = np.array([xp.var( name='v_{0}_{1}'.format(i, j), vartype=xp.binary)
                  for i in curricula for j in timeslots], dtype=xp.npvar).reshape(len(curricula), len(timeslots))


In [ ]:
stage_1.addVariable(x, y, z, w, r, v)

In [ ]:
#Add constraints

#C1: A course i in C should be assigned exactly L(c) lectures that is L(c)== #timeslots(c)
#C1 = [xp.Sum(x[i, j, k] for j in (timeslots)) == lecture_hours_dict[i] for i in (courses) for k in (class_type) if k == 0]
C1 = [xp.Sum(x[i, j, 0] for j in (timeslots)) == lecture_hours[i] for i in range(len(courses))]


**STAGE 2**

In [ ]:
# Create a problem 

stage_2 = xp.problem(name='F_O stage 2')

In [ ]:
# Create the variables

#u = np.array([xp.var( name='u_{0}_{1}_{2}_{3}'.format(i, j, k, l), vartype=xp.binary)
#                  for i in courses for j in timeslots for k in rooms for l in class_type], dtype=xp.npvar).reshape(len(courses), len(timeslots), len(rooms), len(class_type))
u = np.array([xp.var( name='u_{0}_{1}_{2}'.format(i, j, k), vartype=xp.binary)
                  for i in courses for j in timeslots for k in rooms], dtype=xp.npvar).reshape(len(courses), len(timeslots), len(rooms))

t = np.array([xp.var( name='t_{0}_{1}'.format(i, j), vartype=xp.binary)
                  for i in courses for j in rooms], dtype=xp.npvar).reshape(len(courses), len(rooms))




In [ ]:
# Add the variables to the problem
stage_2.addVariable(u, t)

In [ ]:
# Constraints
# Remeber to change c_2 in accordance to solution from stage 1

#b_2 = [xp.Sum(u[i, j, k, l] for j in timeslots) - len(timeslots) * t[i, k] <= 0 for i in range(len(courses)) for k in range(len(rooms)) for l in range(len(class_type))]
#c_2 = [xp.Sum(u[i, j, k, l] for k in range(len(rooms))) == 1 for i in range(len(courses)) for j in timeslots for l in range(len(class_type))]
#d_2 = [xp.Sum(u[i, j, k, l] for i in range(len(courses)) for j in timeslots) <= 1 for k in  range(len(rooms)) for l in range(len(class_type))]
b_2 = [xp.Sum(u[i, j, k] for j in timeslots) - len(timeslots) * t[i, k] <= 0 for i in range(len(courses)) for k in range(len(rooms))]
c_2 = [xp.Sum(u[i, j, k] for k in range(len(rooms))) == 1 for i in range(len(courses)) for j in timeslots]
d_2 = [xp.Sum(u[i, j, k] for i in range(len(courses)) for j in timeslots) <= 1 for k in  range(len(rooms))]



In [ ]:
stage_2.addConstraint(b_2, c_2, d_2)

In [ ]:
stage_2.setObjective(xp.Sum(t[i, j] for i in range(len(courses)) for j in range(len(rooms))), sense = xp.minimize)
stage_2.solve()